## 네이버 영화평 감성분석 - LSTM

In [13]:
!pip install Konlpy > /dev/null

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [5]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


#### 1. 데이터 전처리
- train dataset

In [6]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [14]:
# 결측치 데이터 삭제
train_df.dropna(how='any', inplace=True) # 결측치가 하나라도 존재하는 행 삭제
train_df.shape

(149995, 3)

In [15]:
train_df.document.nunique()

146182

In [16]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

# subset=['document']: 중복을 확인할 때 'document'열의 값을 기준으로 중복을 제거

(146182, 3)

In [17]:
# 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- test dataset

In [18]:
test_df.dropna(how='any', inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49157, 3)

In [19]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

#### 2. 텍스트 전처리
- train dataset

In [20]:
# 한글을 제외한 모든 문자를 공백으로 대체
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [21]:
# 한글이 없는 글 --> ''만 남게됨
# ''만 남은 데이터는 제거: np.nan으로 대체후 dropna 실행
train_df.document.replace('', np.nan, inplace=True)
train_df.document.isna().sum()

789

In [22]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145393, 3)

- test dataset

In [23]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
test_df.document.replace('', np.nan, inplace=True) # 빈 문자열 -> np.nan으로 대체
test_df.dropna(how='any', inplace=True)
test_df.shape

(48852, 3)

#### 3. 한글 형태소 분석


In [24]:
from konlpy.tag import Okt
okt = Okt()

In [25]:
from google.colab import files
up = files.upload()

Saving 한글불용어100.txt to 한글불용어100 (2).txt


In [11]:
with open('한글불용어100.txt') as st:
    lines = st.readlines()

# 'line' 리스트의 각 요소를 순회하면서 탭('\t')문자를 기준으로 분리한 후 첫 번째 요소를 추출
stop_words = [line.split('\t')[0] for line in lines]
stop_words[:10]

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보']

In [26]:
from tqdm import tqdm # 진행 상황을 시각화. 'for'문에 사용되어 반복이 진행되는 동안 프로그래스 바를 표시

X_train = [] # 불용어가 제거된 형태소로 분해된 리뷰를 담을 빈 리스트 생성
for review in tqdm(train_df.document):
    morphs = okt.morphs(review, stem=True) # stem=True: 형태소 분석 시 어간 추출을 수행하도록 지정
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_train.append(clean_morph_review)

100%|██████████| 145393/145393 [10:17<00:00, 235.50it/s]


In [27]:
%%time
X_test = []
for review in test_df.document:
    morphs = okt.morphs(review, stem=True)
    clean_morph_review = ' '.join([morph for morph in morphs if morph not in stop_words])
    X_test.append(clean_morph_review)

CPU times: user 3min 5s, sys: 658 ms, total: 3min 6s
Wall time: 3min 4s


#### 4. Keras Tokenizer

In [ ]:
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
t = Tokenizer()
t.fit_on_texts(X_train)
len(t.word_index)

In [ ]:
# 빈도수 상위 10,000개 단어로 인코딩
num_words = 10000
t = Tokenizer(num_words=num_words)
t.fit_on_texts(X_train)

In [ ]:
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

In [ ]:
# 데이터의 최대/평균 길이
max(len(s) for s in X_train), sum(len(s) for s in X_train) / len(X_train)

In [ ]:
max_len = 20 # 패딩 작업을 수행할 때 사용할 최대 길이

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
y_train = train_df.label.values
y_test = test_df.label.values

#### 5. LSTM 모델 정의/설정/학습

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    LSTM(128),
    Dense(1, 'sigmoid')
])
model.summary()

In [ ]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])

model_path = 'best_naver_movie_lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

In [ ]:
hist = model.fit(
    X_train, y_train, validation_split=0.2, epochs=30, batch_size=128, callbacks=[mc,es]
)

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)